In [ ]:
import tkinter as tk
from tkinter import messagebox
import pygame
import yt_dlp
import os
import threading

# Initialize Pygame
pygame.mixer.init()

# Folder to save downloaded songs
DOWNLOAD_DIR = "downloaded_songs"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# GUI setup
root = tk.Tk()
root.title("🎵 Online Music Search & Player")
root.geometry("500x450")
root.config(bg="#111111")

# Fonts and Colors
FONT = ("Segoe UI", 12)
GREEN = "#1DB954"
DARK = "#191414"

# Song path
current_song = ""

# Search and download song
def search_and_play():
    query = entry.get().strip()
    if not query:
        messagebox.showerror("Error", "Please enter a song name.")
        return

    status_label.config(text="🔍 Searching...", fg="white")
    threading.Thread(target=download_and_play, args=(query,)).start()

def download_and_play(query):
    global current_song

    try:
        filename = os.path.join(DOWNLOAD_DIR, query.replace(" ", "_") + ".mp3")
        ydl_opts = {
            'format': 'bestaudio/best',
            'quiet': True,
            'noplaylist': True,
            'outtmpl': filename,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f"ytsearch1:{query}"])

        current_song = filename
        pygame.mixer.music.load(current_song)
        pygame.mixer.music.play()
        song_label.config(text="Now playing: " + os.path.basename(current_song))
        status_label.config(text="✅ Playing...", fg=GREEN)

    except Exception as e:
        status_label.config(text="❌ Failed to load song", fg="red")
        messagebox.showerror("Error", str(e))

# Controls
def pause(): pygame.mixer.music.pause()
def resume(): pygame.mixer.music.unpause()
def stop(): pygame.mixer.music.stop()

# UI Elements
tk.Label(root, text="🎧 Online Spotify-Like Player", font=("Segoe UI", 16, "bold"), bg="#111111", fg=GREEN).pack(pady=20)
entry = tk.Entry(root, font=FONT, width=40)
entry.pack(pady=10)

tk.Button(root, text="🔍 Search & Play", font=FONT, command=search_and_play,
          bg=DARK, fg="white", activebackground=GREEN, activeforeground="black").pack(pady=10)

# Song display
song_label = tk.Label(root, text="No song loaded", font=FONT, bg="#111111", fg="white")
song_label.pack(pady=10)

# Controls
btn_frame = tk.Frame(root, bg="#111111")
btn_frame.pack(pady=20)

for i, (text, cmd) in enumerate([
    ("⏸️ Pause", pause),
    ("▶️ Resume", resume),
    ("⏹️ Stop", stop)
]):
    tk.Button(btn_frame, text=text, font=FONT, command=cmd,
              bg=DARK, fg="white", activebackground=GREEN,
              activeforeground="black", width=12).grid(row=0, column=i, padx=10)

# Status
status_label = tk.Label(root, text="", font=("Segoe UI", 10), bg="#111111", fg="gray")
status_label.pack(pady=10)

tk.Label(root, text="Built with ❤️ using Python & yt-dlp", font=("Segoe UI", 9), bg="#111111", fg="gray").pack(side="bottom", pady=10)

root.mainloop()


In [3]:
import tkinter as tk
from tkinter import ttk
import yt_dlp
import vlc
import threading

player = None
volume_level = 100
is_video = False
is_playing = False

def stream_content():
    global player, volume_level, is_video, is_playing
    query = entry.get().strip()
    is_video = mode_var.get() == "video"

    if not query:
        status_label.config(text="❌ Enter song name.", fg="red")
        return

    status_label.config(text="🔍 Searching...", fg="white")

    def run():
        global player, is_playing
        try:
            ydl_opts = {
                'format': 'best' if is_video else 'bestaudio/best',
                'quiet': True,
                'noplaylist': True,
                'extract_flat': False,
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(f"ytsearch1:{query}", download=False)['entries'][0]
                media_url = info['url']
                title = info.get("title", "Unknown Title")

            if player:
                player.stop()

            instance = vlc.Instance()
            player = instance.media_player_new()
            media = instance.media_new(media_url)
            player.set_media(media)

            if is_video:
                player.set_fullscreen(True)

            player.audio_set_volume(volume_level)
            player.play()
            is_playing = True

            status_label.config(
                text=f"🎬 Playing Video: {title}" if is_video else f"🎧 Playing Audio: {title}",
                fg="#1DB954"
            )

            update_progress()

        except Exception as e:
            status_label.config(text=f"❌ Error: {str(e)}", fg="red")

    threading.Thread(target=run, daemon=True).start()

def stop_playback():
    global player, is_playing
    if player:
        player.stop()
        is_playing = False
        progress_var.set(0)
        status_label.config(text="⏹️ Stopped", fg="white")

def pause_song():
    if player:
        player.pause()
        status_label.config(text="⏸️ Paused", fg="white")

def resume_song():
    if player:
        player.play()
        status_label.config(text="▶️ Resumed", fg="white")

def volume_changed(val):
    global volume_level
    volume_level = int(float(val))
    if player:
        player.audio_set_volume(volume_level)
        status_label.config(text=f"🔊 Volume: {volume_level}%", fg="white")

def update_progress():
    def loop():
        while is_playing:
            if player and player.get_length() > 0:
                current = player.get_time()
                total = player.get_length()
                if total > 0:
                    progress = int((current / total) * 100)
                    progress_var.set(progress)
            root.after(1000, lambda: None)
    threading.Thread(target=loop, daemon=True).start()

def on_close():
    global player
    if player:
        player.stop()
    root.destroy()

# ---------------- GUI ---------------- #
root = tk.Tk()
root.title("🎵 VibeStream – YouTube Player")
root.geometry("650x560")
root.config(bg="#0e0e0e")

FONT = ("Segoe UI", 12)
BIG = ("Segoe UI", 14, "bold")
GREEN = "#1DB954"
DARK = "#191414"
WHITE = "white"

# Title
tk.Label(root, text="🎧 VibeStream", font=("Segoe UI", 20, "bold"), bg="#0e0e0e", fg=GREEN).pack(pady=15)

# Entry
entry = tk.Entry(root, font=FONT, width=40, bg=DARK, fg=WHITE, insertbackground=WHITE, relief="flat", highlightthickness=2, highlightbackground=GREEN)
entry.pack(pady=10)

# Audio/Video Mode
mode_var = tk.StringVar(value="audio")
mode_frame = tk.Frame(root, bg="#0e0e0e")
tk.Radiobutton(mode_frame, text="🎵 Audio Only", variable=mode_var, value="audio", font=FONT, bg="#0e0e0e", fg="white", selectcolor=GREEN).pack(side="left", padx=20)
tk.Radiobutton(mode_frame, text="🎥 Full Video", variable=mode_var, value="video", font=FONT, bg="#0e0e0e", fg="white", selectcolor=GREEN).pack(side="left", padx=20)
mode_frame.pack(pady=5)

# Play Button
tk.Button(root, text="▶️ Play", font=BIG, command=stream_content, bg=GREEN, fg=DARK, width=28, height=2, relief="flat", bd=0).pack(pady=10)

# Controls
ctrl_frame = tk.Frame(root, bg="#0e0e0e")
ctrl_frame.pack(pady=15)

controls = [
    ("⏸ Pause", pause_song),
    ("▶️ Resume", resume_song),
    ("⏹ Stop", stop_playback),
]

for i, (label, cmd) in enumerate(controls):
    tk.Button(ctrl_frame, text=label, font=FONT, command=cmd, bg=DARK, fg=WHITE, activebackground=GREEN, activeforeground=DARK, width=14, relief="flat", bd=0).grid(row=0, column=i, padx=10, pady=10)

# Volume slider
tk.Label(root, text="🔊 Volume", font=FONT, bg="#0e0e0e", fg=WHITE).pack()
volume_slider = tk.Scale(root, from_=0, to=200, orient="horizontal", command=volume_changed, bg="#0e0e0e", fg=WHITE, troughcolor=DARK, highlightthickness=0)
volume_slider.set(volume_level)
volume_slider.pack(pady=5)

# Progress bar
progress_var = tk.IntVar()
tk.Label(root, text="⏱ Progress", font=FONT, bg="#0e0e0e", fg=WHITE).pack(pady=5)
progress_bar = ttk.Progressbar(root, variable=progress_var, length=400)
progress_bar.pack(pady=5)

# Status
status_label = tk.Label(root, text="🎵 Ready", font=("Segoe UI", 11), bg="#0e0e0e", fg="gray")
status_label.pack(pady=20)

tk.Label(root, text="Built with ❤️ using VLC + yt-dlp", font=("Segoe UI", 9), bg="#0e0e0e", fg="gray").pack(side="bottom", pady=10)

# Exit handler
root.protocol("WM_DELETE_WINDOW", on_close)
root.mainloop()
